In [1]:
import duckdb  # Importa o DuckDB para manipulação de dados e execução de SQL
import os  # Importa o módulo os para interagir com variáveis de ambiente do sistema
from dotenv import load_dotenv, find_dotenv  # Importa funções para carregar variáveis de ambiente de um arquivo .env
from datetime import datetime
import pandas as pd
from deltalake import DeltaTable, write_deltalake
# Carrega as variáveis de ambiente definidas no arquivo .env
load_dotenv(find_dotenv())

# Define as credenciais de acesso ao MinIO a partir das variáveis de ambiente
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_MINIO")  # Chave de acesso do MinIO
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY_MINIO")  # Chave secreta do MinIO
HOST_MINIO = os.getenv("HOST_MINIO")  # Host do MinIO

# Conecta ao DuckDB, criando uma instância de conexão
con = duckdb.connect()

# Cria uma secret no DuckDB para acessar o MinIO com as credenciais fornecidas
con.execute(f"""
    CREATE SECRET my_minio_secret (
        TYPE 'S3',
        KEY_ID '{AWS_ACCESS_KEY}',
        SECRET '{AWS_SECRET_KEY}',
        REGION 'us-east-1',
        ENDPOINT '{HOST_MINIO}:9000',
        URL_STYLE 'path',
        USE_SSL false
    );
""")

# Define o caminho de destino no MinIO para os arquivos Parquet
path_minio_landing = 's3://landing/comex'
path_minio_bronze = 's3://bronze/comex/ncm_fat_agreg'

In [2]:
storage_options = {
    "AWS_ENDPOINT_URL": f"http://{HOST_MINIO}:9000",
    "AWS_REGION": "us-east-1",
    "AWS_ACCESS_KEY_ID": AWS_ACCESS_KEY,
    "AWS_SECRET_ACCESS_KEY": AWS_SECRET_KEY,
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true",
    "AWS_ALLOW_HTTP": "true"
}

In [3]:
con.sql(f"""
        SELECT 
        *
         from '{path_minio_landing}/NCM_FAT_AGREG.parquet';        

        """)

┌──────────────┬────────────────────────────┬───────────────────────────┐
│ CO_FAT_AGREG │        NO_FAT_AGREG        │      NO_FAT_AGREG_GP      │
│   varchar    │          varchar           │          varchar          │
├──────────────┼────────────────────────────┼───────────────────────────┤
│ 01           │ PRODUTOS BASICOS           │ PRODUTOS BASICOS          │
│ 02           │ PRODUTOS SEMIMANUFATURADOS │ PRODUTOS INDUSTRIALIZADOS │
│ 03           │ PRODUTOS MANUFATURADOS     │ PRODUTOS INDUSTRIALIZADOS │
│ 04           │ TRANSACOES ESPECIAIS       │ OPERACOES ESPECIAIS       │
│ 05           │ CONSUMO DE BORDO           │ OPERACOES ESPECIAIS       │
│ 06           │ REEXPORTACAO               │ OPERACOES ESPECIAIS       │
└──────────────┴────────────────────────────┴───────────────────────────┘

In [4]:
df = con.sql(f"""
        SELECT 
        CAST(CO_FAT_AGREG  as int) AS cod_fator_agregado,
        NO_FAT_AGREG AS nome_fator_agregado,
        NO_FAT_AGREG_GP AS grupo_fator_agregado
         from '{path_minio_landing}/NCM_FAT_AGREG.parquet';        
        """).to_arrow_table()

In [5]:
df.to_pandas()

pyarrow.Table
cod_fator_agregado: int32
nome_fator_agregado: string
grupo_fator_agregado: string
----
cod_fator_agregado: [[1,2,3,4,5,6]]
nome_fator_agregado: [["PRODUTOS BASICOS","PRODUTOS SEMIMANUFATURADOS","PRODUTOS MANUFATURADOS","TRANSACOES ESPECIAIS","CONSUMO DE BORDO","REEXPORTACAO"]]
grupo_fator_agregado: [["PRODUTOS BASICOS","PRODUTOS INDUSTRIALIZADOS","PRODUTOS INDUSTRIALIZADOS","OPERACOES ESPECIAIS","OPERACOES ESPECIAIS","OPERACOES ESPECIAIS"]]

In [6]:
write_deltalake(
        f'{path_minio_bronze}',
        df,
        storage_options=storage_options,
        
    )

In [7]:
table_path = f'{path_minio_bronze}'

# Conecte à tabela Delta existente
table = DeltaTable(table_path, storage_options=storage_options)



In [8]:
table.merge(
    source=df,
    predicate='target.cod_fator_agregado = source.cod_fator_agregado',
    source_alias="source",
    target_alias="target",
).when_not_matched_insert_all().execute()


{'num_source_rows': 6,
 'num_target_rows_inserted': 0,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 0,
 'num_target_files_added': 0,
 'num_target_files_removed': 0,
 'execution_time_ms': 67,
 'scan_time_ms': 0,
 'rewrite_time_ms': 26}

In [9]:
con.sql(f"""
        SELECT *
FROM delta_scan('{path_minio_bronze}')
        """)

┌────────────────────┬────────────────────────────┬───────────────────────────┐
│ cod_fator_agregado │    nome_fator_agregado     │   grupo_fator_agregado    │
│       int32        │          varchar           │          varchar          │
├────────────────────┼────────────────────────────┼───────────────────────────┤
│                  1 │ PRODUTOS BASICOS           │ PRODUTOS BASICOS          │
│                  2 │ PRODUTOS SEMIMANUFATURADOS │ PRODUTOS INDUSTRIALIZADOS │
│                  3 │ PRODUTOS MANUFATURADOS     │ PRODUTOS INDUSTRIALIZADOS │
│                  4 │ TRANSACOES ESPECIAIS       │ OPERACOES ESPECIAIS       │
│                  5 │ CONSUMO DE BORDO           │ OPERACOES ESPECIAIS       │
│                  6 │ REEXPORTACAO               │ OPERACOES ESPECIAIS       │
└────────────────────┴────────────────────────────┴───────────────────────────┘

In [10]:
con.close()